In [1]:
import pandas as pd
import urllib.parse
import sqlalchemy as sa
from sqlalchemy import create_engine
import os

## Переменные для подключения к БД

Читаем данные для подключения из файла:

In [2]:
file = open('db_user.txt', 'r')

In [3]:
# данные в файле прописаны строкой, преобразовываем в словарь
file = eval(file.read())

Прописываем переменные для подключения к БД:

In [4]:
user = file.get('login')
password = urllib.parse.quote_plus(file.get('password')) #Экранируем все специальные символы в пароле (необходимо для подключения в модуле engine)
host = file.get('host')
port = file.get('port')
db = file.get('db')

## Функции для работы с БД

### Чтение данных

#### Создание подключения к БД

In [5]:
def create_db_config_and_connect(user, password, host, port, db):
    '''
    Получает константы для подключения к БД, создает объект с конфигурацией БД.
    Формирует строку подключения и возвращает объект подключения к БД
    '''
    # записываем конфигурацию БД
    db_config = {
        'user': user, # имя пользователя
        'pwd': password, # пароль
        'host': host,
        'port': port, # порт подключения
        'db': db # название базы данных
    }
    # формируем строку подключения
    connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
        db_config['user'],
        db_config['pwd'],
        db_config['host'],
        db_config['port'],
        db_config['db'],
    )
    # создаем строку подключения к БД
    engine = create_engine(connection_string)
    return engine

Пример работы:

In [6]:
engine = create_db_config_and_connect(user, password, host, port, db)

*Строка подключения не отображается из соображений безопасности*

#### Обработка исключений

In [8]:
# обрабатываемые исключения
data_reading_error = 'Не удалось прочитать данные'
db_connection_error = 'Не удалось подключиться к БД'
db_table_error = 'Не найдена таблица БД'
data_record_error = 'Не удалось выполнить запрос на запись данных'
record_confirmation_error = 'Не удалось подтвердить запись данных'

In [9]:
def catch_exceptions(handled_type, exception):
    print(handled_type)
    print("-"*60)
    print(f"Тип исключения: {type(exception).__name__}, сообщение: {str(exception)}")
    print("-"*60)
    return

#### Получение данных из БД

In [10]:
def read_sql_query(query, engine):
    '''
    Получает запрос для чтения данных и объект соединения с БД.
    Подключается к БД, возвращает результат чтения данных из БД
    '''
    try:
        result = pd.read_sql_query(query, con = engine)
    except Exception as e:
        handled_type = data_reading_error
        catch_exceptions(handled_type, e)
    return result

##### Пример работы без ошибок:

In [13]:
query = '''
SELECT * FROM quick_filters
LIMIT 1
'''

In [14]:
read_sql_query(query, engine)

,code,enabled,name
0,LUKOIL_GAS_STATION,True,АЗС Лукойл


##### Пример работы с ошибкой:

In [15]:
query = '''
SELECT * FROM some_disappeared_table
LIMIT 1
'''

In [16]:
read_sql_query(query, engine)

Не удалось прочитать данные
------------------------------------------------------------
Тип исключения: ProgrammingError, сообщение: (psycopg2.errors.UndefinedTable) relation "some_disappeared_table" does not exist
LINE 2: SELECT * FROM some_disappeared_table
                      ^

[SQL: 
SELECT * FROM some_disappeared_table
LIMIT 1
]
(Background on this error at: https://sqlalche.me/e/20/f405)
------------------------------------------------------------


UnboundLocalError: cannot access local variable 'result' where it is not associated with a value

### Запись данных в БД без создания таблицы (в существующую)

#### С выводом количества строк в таблице до и после записи

Подсчитать количество строк в таблице нужно дважды: перед записью данных и после. Поэтому используем функцию для подсчета записей:

In [17]:
def count_table_rows(table,engine):
    '''
    Принимает на вход название таблицы строкой и объект соединения.
    Возвращает количество строк в таблице
    '''
    query = f'''
    SELECT COUNT(*) AS rows_count FROM {table}
    '''
    rows = read_sql_query(query, engine)
    return rows['rows_count'][0]

##### Пример работы

*Обработка ошибок наследуется из функции чтения данных*

In [18]:
table = 'quick_filters'

In [20]:
count_table_rows(table,engine)

5

In [21]:
def connection_to_db_and_insert(engine, table, values_to_insert):
    '''
    Подключается к БД, создает запрос для вставки данных в нужную таблицу.
    Выполняет запрос с подключением и возвращает результат
    '''
    # подключение к БД и загрузка объекта MetaData
    try:
        conn = engine.connect()
        metadata = sa.MetaData()
    except Exception as e: # Обработка ошибок с подключением к БД
        handled_type = db_connection_error
        catch_exceptions(handled_type, e)
    # подключение к таблице в БД
    try:
        table_to_insert = sa.Table(table, metadata, autoload_with=engine)
    except Exception as e: # Обработка ошибок с подключением к таблице в БД
        handled_type = db_table_error
        catch_exceptions(handled_type, e)
    print('Количество строк до записи:', count_table_rows(table,engine))
    # формируем запрос для вставки данных
    insertion_query = table_to_insert.insert().values(values_to_insert)
    # выполнение и подтверждение запроса
    try:
        conn.execute(insertion_query)
    except Exception as e: # Обработка ошибки записи
        handled_type = data_record_error
        catch_exceptions(handled_type, e)
    try:
        conn.commit()
    except Exception as e: # Обработка ошибки подтверждения записи
        handled_type = record_confirmation_error
        catch_exceptions(handled_type, e)
    # проверка результата записи
    print('Количество строк после записи:', count_table_rows(table,engine))  
    # закрытие соединения
    conn.close()
    return

##### Пример работы без ошибок

In [22]:
values_to_insert = [{'code': 'SILVER', 'enabled': False, 'name': 'Серебряные'}]

In [23]:
connection_to_db_and_insert(engine, table, values_to_insert)

Количество строк до записи: 5
Количество строк после записи: 6


##### Пример работы с ошибкой:

In [24]:
values_to_insert = [{'codeeeeee': 'SILVER', 'enabled': False, 'name': 'Серебряные'}]

In [25]:
connection_to_db_and_insert(engine, table, values_to_insert)

Количество строк до записи: 5
Не удалось выполнить запрос на запись данных
------------------------------------------------------------
Тип исключения: CompileError, сообщение: Unconsumed column names: codeeeeee
------------------------------------------------------------
Количество строк после записи: 5


C:\Users\Licard\AppData\Local\Temp\ipykernel_15048\3437858971.py:24: SAWarning: Column 'quick_filters.code' is marked as a member of the primary key for table 'quick_filters', but has no Python-side or server-side default generator indicated, nor does it indicate 'autoincrement=True' or 'nullable=True', and no explicit value is passed.  Primary key columns typically may not store NULL.
  conn.execute(insertion_query)
